<a href="https://colab.research.google.com/github/mahinuralam/notebooks/blob/main/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define the input shape and number of classes (Make sure to use the same values as in the training)
input_shape = (224, 224, 3)
num_classes = 5  # Update this line with the correct number of classes in your dataset
# Adjust the batch size, learning rate, and augmentation parameters
batch_size = 32
learning_rate = 0.0001


# Define the path to the Downloads folder
train_data_dir = os.path.expanduser('~/Downloads/labeled_dataset_loco')

# Create data generators for training and validation with augmentation
train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.4
)

train_generator = train_data_gen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='sparse',  # Update class_mode to 'sparse'
    subset='training'
)

val_generator = train_data_gen.flow_from_directory(
    train_data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='sparse',  # Update class_mode to 'sparse'
    subset='validation'
)

# Define the path to the labeled dataset directory
test_data_dir = os.path.expanduser('~/Downloads/labeled_dataset_loco')  # Update this with the path to your labeled dataset

# Create a data generator for testing
test_data_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_data_gen.flow_from_directory(
    test_data_dir,
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='sparse',  # Use 'sparse' for numerical class labels
    shuffle=False,
)


In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average
import tensorflow as tf

DenseNet121 = tf.keras.models.load_model('/home/mahin/Documents/notebook/Anomaly/DenseNet121.h5')
DenseNet121 = Model(inputs=DenseNet121.inputs,
                outputs=DenseNet121.outputs,
                name='DenseNet121')

EfficientNetB0 = load_model('/home/mahin/Documents/notebook/Anomaly/EfficientNetB0.h5')
EfficientNetB0 = Model(inputs=EfficientNetB0.inputs,
                outputs=EfficientNetB0.outputs,
                name='EfficientNetB0')

InceptionV3 = load_model('/home/mahin/Documents/notebook/Anomaly/InceptionV3.h5')
InceptionV3 = Model(inputs=InceptionV3.inputs,
                outputs=InceptionV3.outputs,
                name='InceptionV3')

ResNet50 = load_model('/home/mahin/Documents/notebook/Anomaly/ResNet50.h5')
ResNet50 = Model(inputs=ResNet50.inputs,
                outputs=ResNet50.outputs,
                name='ResNet50')

VGG16 = load_model('/home/mahin/Documents/notebook/Anomaly/VGG16.h5')
VGG16 = Model(inputs=VGG16.inputs,
                outputs=VGG16.outputs,
                name='VGG16')

Xception = load_model('/home/mahin/Documents/notebook/Anomaly/Xception.h5')
Xception = Model(inputs=Xception.inputs,
                outputs=Xception.outputs,
                name='Xception')


models = [DenseNet121, EfficientNetB0, InceptionV3, ResNet50, VGG16, Xception]

model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

ensemble_model.compile(optimizer='adam',loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

history=ensemble_model.fit(
  train_generator,
  validation_data=val_generator,
  epochs=25,
)




In [ ]:
import os
print(os.getcwd())


In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average
from tensorflow.keras.applications import DenseNet121, EfficientNetB0, InceptionV3, ResNet50, VGG16, Xception
import tensorflow as tf

# Define paths to the model weights
subdir = '/home/mahin/Documents/notebook/Anomaly'
dense_path = f'{subdir}/DensNet121.weights.h5'
efficient_path = f'{subdir}/EfficientNetB0.h5'
inception_path = f'{subdir}/InceptionV3.h5'
resnet_path = f'{subdir}/ResNet50.h5'
vgg_path = f'{subdir}/VGG16.h5'
xception_path = f'{subdir}/Xception.h5'

# Create and load models
DenseNet121_model = DenseNet121(input_shape=(224, 224, 3), include_top=True)
DenseNet121_model.load_weights(dense_path)

EfficientNetB0_model = EfficientNetB0(input_shape=(224, 224, 3), include_top=True)
EfficientNetB0_model.load_weights(efficient_path)

InceptionV3_model = InceptionV3(input_shape=(224, 224, 3), include_top=True)
InceptionV3_model.load_weights(inception_path)

ResNet50_model = ResNet50(input_shape=(224, 224, 3), include_top=True)
ResNet50_model.load_weights(resnet_path)

VGG16_model = VGG16(input_shape=(224, 224, 3), include_top=True)
VGG16_model.load_weights(vgg_path)

Xception_model = Xception(input_shape=(224, 224, 3), include_top=True)
Xception_model.load_weights(xception_path)

models = [DenseNet121_model, EfficientNetB0_model, InceptionV3_model, ResNet50_model, VGG16_model, Xception_model]

model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

ensemble_model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

history = ensemble_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,
)


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Average
import tensorflow as tf

# Define paths to the full model files
subdir = '/home/mahin/Documents/notebook/Anomaly'
dense_path = f'{subdir}/DenseNet121.h5'
efficient_path = f'{subdir}/EfficientNetB0.h5'
inception_path = f'{subdir}/InceptionV3.h5'
resnet_path = f'{subdir}/ResNet50.h5'
vgg_path = f'{subdir}/VGG16.h5'
xception_path = f'{subdir}/Xception.h5'

# Load the full models
DenseNet121_model = load_model(dense_path)
EfficientNetB0_model = load_model(efficient_path)
InceptionV3_model = load_model(inception_path)
ResNet50_model = load_model(resnet_path)
VGG16_model = load_model(vgg_path)
Xception_model = load_model(xception_path)

models = [DenseNet121_model, EfficientNetB0_model, InceptionV3_model, ResNet50_model, VGG16_model, Xception_model]

model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

ensemble_model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

history = ensemble_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,
)


In [ ]:
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt


model1 = load_model('DenseNet121.h5')

model2 = load_model('InceptionV3.h5')

models = [model1, model2]

def ensemble_predictions(members, testX):
 yhats = [model.predict(testX) for model in members]
 yhats = np.array(yhats)
 # sum across ensemble members
 summed = np.sum(yhats, axis=0)
 # argmax across classes
 result = np.argmax(summed, axis=1)
 return result


# example of predicting
img = plt.imread(test_dir + file)

# Resize it to the net input size:
img = cv2.resize(img, (224, 224))
img = img[np.newaxis, ...]

# Convert the data to float:
img = img.astype(np.float32)

class_index = ensemble_predictions(models, img)[0]


# Convert class id to name


label = class_names[class_index]





In [ ]:
import os
import numpy as np
import shutil
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout


# Define the input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 5  # Update this line with the correct number of classes in your dataset

# Create the Xception model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Change activation to 'softmax'
ResNet50 = Model(inputs=base_model.input, outputs=predictions)

# Adjust the batch size, learning rate, and augmentation parameters
batch_size = 32
learning_rate = 0.0001

# Compile the model
ResNet50.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])

# model.summary()


from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Create checkpoint callback
checkpoint_path = 'checkpoint/ResNet50.weights.h5'
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      save_weights_only=True,
                                      monitor="val_accuracy",
                                      save_best_only=True)

# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               patience = 5,
                               restore_best_weights = True) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)




# Train the model
history1 = ResNet50.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=35
)

In [ ]:
import os
import numpy as np
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout



# Define the input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 5  # Update this line with the correct number of classes in your dataset

# Create the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Change activation to 'softmax'
VGG16 = Model(inputs=base_model.input, outputs=predictions)

# Adjust the batch size, learning rate, and augmentation parameters
batch_size = 32
learning_rate = 0.0001

# Compile the model
VGG16.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])

# model.summary()

# Train the model
history2 = VGG16.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=35
)

In [ ]:
import os
import numpy as np
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout



# Define the input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 5  # Update this line with the correct number of classes in your dataset

# Create the VGG16 model
base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Change activation to 'softmax'
Xception = Model(inputs=base_model.input, outputs=predictions)

# Adjust the batch size, learning rate, and augmentation parameters
batch_size = 32
learning_rate = 0.0001

# Compile the model
Xception.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])

# model.summary()

# Train the model
history3 = Xception.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=35
)

In [ ]:
import os
import numpy as np
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout



# Define the input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 5  # Update this line with the correct number of classes in your dataset

# Create the VGG16 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Change activation to 'softmax'
InceptionV3 = Model(inputs=base_model.input, outputs=predictions)

# Adjust the batch size, learning rate, and augmentation parameters
batch_size = 32
learning_rate = 0.0001

# Compile the model
InceptionV3.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])

# model.summary()

# Train the model
history4 = InceptionV3.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=35
)

In [ ]:
import os
import numpy as np
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout



# Define the input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 5  # Update this line with the correct number of classes in your dataset

# Create the VGG16 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Change activation to 'softmax'
EfficientNetB0 = Model(inputs=base_model.input, outputs=predictions)

# Adjust the batch size, learning rate, and augmentation parameters
batch_size = 32
learning_rate = 0.0001

# Compile the model
EfficientNetB0.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])

# model.summary()

# Train the model
history4 = EfficientNetB0.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=35

)

In [ ]:
import os
import numpy as np
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout



# Define the input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 5  # Update this line with the correct number of classes in your dataset

# Create the VGG16 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Change activation to 'softmax'
DenseNet121 = Model(inputs=base_model.input, outputs=predictions)

# Adjust the batch size, learning rate, and augmentation parameters
batch_size = 32
learning_rate = 0.0001

# Compile the model
DenseNet121.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])

# model.summary()

# Train the model
history6 = DenseNet121.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=35
)

In [ ]:
from tensorflow.keras.layers import Input, Average


models = [ResNet50, VGG16, Xception, InceptionV3, EfficientNetB0, DenseNet121]

model_input = Input(shape=(224, 224, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')

ensemble_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])

history = ensemble_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=60,
        callbacks=[
        early_stopping,
        checkpoint_callback,
        reduce_lr
    ]
)

In [ ]:
# Plot the training and validation accuracy
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot the training and validation accuracy
plt.plot(range(1, len(train_accuracy) + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

# Plot the training and validation loss using a logarithmic scale
plt.plot(range(1, len(train_loss) + 1), np.log(train_loss), label='Training Loss')
plt.plot(range(1, len(val_loss) + 1), np.log(val_loss), label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Log Loss')
plt.title('Training and Validation Loss (Log Scale)')
plt.legend()
plt.show()

# Create the accuracy plot
plt.figure()
plt.plot(range(1, len(train_accuracy) + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
# Save the accuracy plot as an image
plt.savefig('accuracy_plot.png', bbox_inches='tight')

# Calculate and display the best validation accuracy and loss
best_val_accuracy = max(val_accuracy)
best_val_loss = min(val_loss)
print(f'Best Validation Accuracy: {best_val_accuracy:.4f}')
print(f'Best Validation Loss: {best_val_loss:.4f}')

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Make predictions on the test data
true_labels = test_generator.classes
predictions = ensemble_model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate the anomaly percentage
total_images = len(true_labels)
anomaly_count = np.sum(predicted_labels != true_labels)
anomaly_percentage = (anomaly_count / total_images) * 100

# Print the anomaly percentage and accuracy
accuracy = (1 - (anomaly_count / total_images)) * 100
print(f'Accuracy: {accuracy:.2f}%')
print(f'Anomaly Percentage: {anomaly_percentage:.2f}%')


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# true_labels are the actual labels from your test data
# predicted_labels are the labels predicted by your model

# Generate the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels, normalize='true')

# Visualize the confusion matrix
fig, ax = plt.subplots(figsize=(10, 10))  # Adjust the size as needed
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_generator.class_indices.keys())
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.xticks(rotation=45)  # Rotate labels to avoid overlap
plt.title('Confusion Matrix')
plt.show()
